In [1]:
# ライブラリの読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optimizers
from scipy.stats import norm
from numpy.random import *
from scipy import optimize

np.random.seed(9837)
torch.manual_seed(9837)

In [ ]:
# 多項分布の乱数を生成する関数
def rmnom(pr, n, k, pattern):
    if pattern==1:
        z_id = np.array(np.argmax(np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis], axis=1), dtype="int")
        Z = np.diag(np.repeat(1, k))[z_id, ]
        return z_id, Z
    z_id = np.array(np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1), dtype="int")
    return z_id

# 切断ポアソン分布を生成する関数
def rtpois(mu, a, b, n, flag=0):
    FA = scipy.stats.poisson.cdf(a, mu)
    FB = scipy.stats.poisson.cdf(b, mu)
    x = np.array(scipy.stats.poisson.ppf(np.random.uniform(0, 1, n)*(FB-FA)+FA, mu), dtype="int")
    if flag==1:
        x = torch.Tensor(x)
    return x

# 切断ポアソン分布を高速で生成する関数
def rtpois_lw(mu, a, b, N, flag=0):
    x = np.random.poisson(mu, N)
    index = np.where(x==0)[0]
    x[index] = rtpois(mu[index], a, b, index.shape[0], 0)
    if flag==1:
        x = torch.Tensor(x)
    return x

# ガンマ分布を生成する関数
def Gamma(w1, w2, flag=0):
    x = np.random.gamma(w1, w2)
    if flag==1:
        x = torch.Tensor(x)
    return x

# 学習データの生成

In [ ]:
# データの生成
# データの定義
modes = 3   #モード数
k = 15
d1 = 1500; d2 = 1500
segment1 = 800; segment2 = 400; segment3 = 400 
word1 = 1500; word2 = 750; word3 = 750
d = d1 + d2
segment = segment1 + segment2 + segment3 
word = word1 + word2 + word3

word = 3000

Lambda = np.random.gamma(50.0, 1/0.2, hh)
pt = np.random.poisson(Lambda, hh)   #モードあたりのレコード数
N = np.sum(pt)
k_vec = [np.repeat(1, k1), np.repeat(1, k2), np.repeat(1, k3)]